In [1]:
import numpy as np
import pandas as pd
import os
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict


In [ ]:
#Path on work laptop
usfws = pd.read_csv("C:\Users\rowanconverse\OneDrive - University of New Mexico\CV4Ecology\Prototyping\Data\Labels\train_val_test\20230307_expertconsensuslabels.csv")
dgc = pd.read_csv("C:\Users\rowanconverse\OneDrive - University of New Mexico\CV4Ecology\Prototyping\Data\Labels\train_val_test\20230307_zooniverseconsensuslabels.csv")
seagull = pd.read_csv("C:\Users\rowanconverse\OneDrive - University of New Mexico\CV4Ecology\Prototyping\Data\Labels\train_val_test\20230307_zooniverseconsensuslabels_seagull.csv")

In [2]:
#Path on home laptop
usfws = pd.read_csv("/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/20230307_expertconsensuslabels.csv")
dgc = pd.read_csv("/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/20230307_zooniverseconsensuslabels.csv")
seagull = pd.read_csv("/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/20230307_zooniverseconsensuslabels_seagull.csv")

In [3]:
#CROWD: merge all Zooniverse annotations, remove "Other" annotations, re-code category IDs, clean columns; add temporary "basefile" column
zoo = pd.concat([dgc,seagull])
zoo = zoo.reset_index(drop=True)
zoo = zoo.drop('Unnamed: 0', axis=1)
zoo = zoo[zoo["category"] != 'Other Bird']
categories = {"Crane": 1, 'Goose': 2, 'Duck': 3, 'Seagull': 4}
zoo['category_id'] = zoo['category'].map(categories)
zoo["basefile"] = [x[:-10] for x in zoo['filename']]

In [4]:
#EXPERT: remove all "Other" IDs; re-code category IDs to match Zooniverse; clean columns
usfws = usfws.drop('Unnamed: 0', axis=1)
usfws = usfws[usfws["category"] != 'Other']
mapping = {'Canadian Goose': 'Goose',
           'Sandhill Crane': 'Crane',
           'Mallard': 'Duck',
           'Northern Pintail': 'Duck',
           'American Wigeon': 'Duck',
           'Ringneck': 'Duck',
           "Ruddy": 'Duck',
           "Readhead": "Duck",
           "Snow Goose": "Goose",
           'Other': 'Other',
           'Teal': 'Duck',
           'Gadwall': 'Duck',
           'Northern Shoveler': 'Duck'}
default = 'Duck'
usfws["superclass"] = usfws["category"].map(mapping).fillna(default)
usfws["category_id"] = usfws["superclass"].map(categories)

In [5]:
#CROWD: Class percentages
duck = len(zoo.loc[zoo["category_id"] == 3]) / len(zoo)
goose = len(zoo.loc[zoo["category_id"] == 2]) / len(zoo)
crane = len(zoo.loc[zoo["category_id"] == 1]) / len(zoo)
sea = len(zoo.loc[zoo["category_id"] == 4]) / len(zoo)
print(duck)
print(goose)
print(crane)
print(sea)

0.6265938811260032
0.1009831479746337
0.1749908339104575
0.09743213698890564


In [22]:
#EXPERT: Class percentages 
duck = len(usfws.loc[usfws["category_id"] == 3]) / len(usfws)
goose = len(usfws.loc[usfws["category_id"] == 2]) / len(usfws)
crane = len(usfws.loc[usfws["category_id"] == 1]) / len(usfws)
print(duck)
print(goose)
print(crane)

0.9167028199566161
0.06073752711496746
0.022559652928416486


In [38]:
#EXPERT: For this small dataset, it was better to split manually using the visualizations below
group = usfws.groupby(["filename"]).value_counts(["superclass"])
group
#for key, item in group:
#    print(group.get_group(key), "\n\n")

filename                 superclass
BDA_12C_20181127_1.JPG   Goose          46
                         Duck           30
                         Crane           9
BDA_12C_20181127_2.JPG   Duck          404
                         Goose          19
                         Crane           7
BDA_12C_20181127_3.JPG   Duck          761
                         Goose          24
BDA_18A4_20181106_1.JPG  Duck          163
BDA_18A4_20181106_2.JPG  Duck          353
                         Crane           2
BDA_18A4_20181106_3.JPG  Duck           62
BDA_18A4_20181106_4.JPG  Duck          113
BDA_18A4_20181107_1.JPG  Duck           66
                         Crane          11
BDA_18A4_20181107_2.JPG  Goose          15
                         Duck           10
                         Crane           1
BDA_18A4_20181107_3.JPG  Duck           89
BDA_18A4_20181107_4.JPG  Duck           62
                         Crane          22
mxw_L13_20181215_1.JPG   Goose          36
dtype: int64

In [47]:
#EXPERT: Creating the train/val/test splits, checking no overlap
ptrain = ["BDA_12C_20181127_3.JPG", "BDA_18A4_20181106_2.JPG","BDA_18A4_20181107_1.JPG","BDA_18A4_20181107_2.JPG", "BDA_18A4_20181107_3.JPG", "BDA_18A4_20181107_4.JPG"]
pval = ["BDA_12C_20181127_2.JPG","BDA_18A4_20181106_1.JPG","BDA_18A4_20181106_4.JPG"]
ptest = ["BDA_12C_20181127_1.JPG","BDA_18A4_20181106_3.JPG","mxw_L13_20181215_1.JPG"]

print(len(ptrain))
print(len(pval))
print(len(ptest))
print(bool(set(ptrain) & set(pval)))
print(bool(set(pval) & set(ptest)))
print(bool(set(ptest) & set(ptrain)))

6
3
3
False
False
False


In [48]:
#Saving the splits to csv files

# Define the column name for the class labels
class_column = 'superclass'

# Split the dataframe into separate groups
train_df = usfws[usfws['filename'].isin(ptrain)]
val_df = usfws[usfws['filename'].isin(pval)]
test_df = usfws[usfws['filename'].isin(ptest)]

# Save the splits to separate CSV files
save = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/usfws_splits/" 
train_df.to_csv(save+'train.csv', index=False)
val_df.to_csv(save+'val.csv', index=False)
test_df.to_csv(save+'test.csv', index=False)

# Calculate the count of labels for each class in each group
train_counts = usfws[usfws['filename'].isin(ptrain)][class_column].value_counts()
val_counts = usfws[usfws['filename'].isin(pval)][class_column].value_counts()
test_counts = usfws[usfws['filename'].isin(ptest)][class_column].value_counts()

# Print the count of labels for each class in each group
print('Train group class counts:')
print(train_counts)

print('\nValidation group class counts:')
print(val_counts)

print('\nTest group class counts:')
print(test_counts)
print("\n total labels: ")
print(test_counts+train_counts+val_counts)


Train group class counts:
Duck     1341
Goose      39
Crane      36
Name: superclass, dtype: int64

Validation group class counts:
Duck     680
Goose     19
Crane      7
Name: superclass, dtype: int64

Test group class counts:
Duck     92
Goose    82
Crane     9
Name: superclass, dtype: int64

 total labels: 
Duck     2113
Goose     140
Crane      52
Name: superclass, dtype: int64


In [8]:
#SPLIT ZOONIVERSE

# Define the desired class distribution in the train, validation, and test sets
class_distribution = {'Crane': 0.18, 'Goose': 0.10, 'Duck': 0.62, 'Seagull': 0.10}

# Calculate the total number of images in the dataset
total_images = zoo['basefile'].nunique()

# Calculate the desired number of images for each class in the train, validation, and test sets
desired_class_counts = {cls: int(count * total_images) for cls, count in class_distribution.items()}

# Create a dictionary to store the assigned images for each set and class
assigned_images = defaultdict(dict)

# Shuffle the unique images
unique_images = zoo['basefile'].unique()
np.random.shuffle(unique_images)

# Assign images to sets while maintaining class distribution
for image in unique_images:
    image_df = zoo[zoo['basefile'] == image]
    classes = image_df['category'].unique()
    
    # Calculate the available class counts for the image
    available_class_counts = {
        cls: desired_class_counts[cls] - assigned_images['train'].get(cls, 0) - assigned_images['val'].get(cls, 0) - assigned_images['test'].get(cls, 0)
        for cls in classes
    }
    
    # Determine the set to assign the image based on available class counts
    available_sets = ['train', 'val', 'test']
    np.random.shuffle(available_sets)
    
    assigned = False
    for set_name in available_sets:
        set_df = zoo[zoo['basefile'].isin(assigned_images[set_name])]
        set_class_counts = set_df['category'].value_counts().to_dict()
        
        if all(available_class_counts[cls] <= set_class_counts.get(cls, 0) for cls in classes):
            assigned_images[set_name][image] = True
            assigned = True
            break
    
    # If the image couldn't be assigned to any set, assign it to the set with the fewest images
    if not assigned:
        set_name = min(available_sets, key=lambda x: len(assigned_images[x]))
        assigned_images[set_name][image] = True

# Split the original dataframe based on the assigned sets
train_df = zoo[zoo['basefile'].isin(assigned_images['train'])]
val_df = zoo[zoo['basefile'].isin(assigned_images['val'])]
test_df = zoo[zoo['basefile'].isin(assigned_images['test'])]

# Check the class distribution in each split
print('Training set class distribution:')
print(train_df['category'].value_counts(normalize=True))

print('\nValidation set class distribution:')
print(val_df['category'].value_counts(normalize=True))

print('\nTest set class distribution:')
print(test_df['category'].value_counts(normalize=True))


Training set class distribution:
Duck       0.534617
Crane      0.379663
Goose      0.084681
Seagull    0.001039
Name: category, dtype: float64

Validation set class distribution:
Duck       0.701394
Seagull    0.189434
Goose      0.106277
Crane      0.002896
Name: category, dtype: float64

Test set class distribution:
Duck       0.707356
Seagull    0.166622
Goose      0.124338
Crane      0.001684
Name: category, dtype: float64


In [19]:
#Double check no overlap between train/val/test
trainlist = set(train_df["basefile"].unique())
vallist = set(val_df["basefile"].unique())
testlist = set(test_df["basefile"].unique())

print(bool(set(testlist) & set(trainlist)))
print(bool(set(testlist) & set(vallist)))
print(bool(set(vallist) & set(trainlist)))

False
False
False


In [20]:
#Save the splits to separate CSV files if desired
save = "/Users/rowanconverse/Library/CloudStorage/OneDrive-UniversityofNewMexico/CV4Ecology/Prototyping/Data/Labels/train_val_test/usfws_splits/" 
train_df.to_csv(save+'train.csv', index=False)
val_df.to_csv(save+'val.csv', index=False)
test_df.to_csv(save+'test.csv', index=False)